# Decorators and Context Managers
Decorators adalah fitur dalam Python yang memungkinkan kita menambahkan atau memodifikasi perilaku fungsi tanpa mengubah kode aslinya. Decorators biasanya digunakan untuk logging, validasi, caching, atau pengukuran waktu eksekusi. Decorators bekerja dengan membungkus fungsi lain dan mengembalikan fungsi baru yang telah dimodifikasi.

Context managers digunakan untuk mengelola sumber daya eksternal, seperti file, koneksi database, atau sesi jaringan. Dengan menggunakan with statement, context manager memastikan bahwa sumber daya dibuka dan ditutup dengan benar, bahkan jika terjadi error. Ini membantu mencegah kebocoran sumber daya dan membuat kode lebih bersih dan aman.

# Simple


## Contoh 1: Eksekusi Fungsi Pengaturan Waktu dengan decorator

Decorator ini mengukur dan mencetak waktu eksekusi dari fungsi yang dibungkus. Cocok digunakan untuk mengetahui performa fungsi.

In [18]:
import time

# Decorator untuk mengukur waktu eksekusi fungsi
def pengukur_waktu(fungsi):
    def bungkus(*args, **kwargs):
        waktu_mulai = time.time()
        hasil = fungsi(*args, **kwargs)
        waktu_selesai = time.time()
        print(f"Fungsi {fungsi.__name__} membutuhkan {waktu_selesai - waktu_mulai} detik untuk dieksekusi.")
        return hasil
    return bungkus

@pengukur_waktu
def fungsi_contoh():
    return 500 ** 2

# Panggil fungsi
fungsi_contoh()

Fungsi fungsi_contoh membutuhkan 7.152557373046875e-07 detik untuk dieksekusi.


250000

## Contoh 2: Eksekusi Fungsi Logging dengan decorator

Decorator ini mencetak argumen yang dikirim ke fungsi dan hasil yang dikembalikan. Berguna untuk debugging dan pelacakan fungsi.

In [19]:
# Decorator untuk mencatat pemanggilan fungsi dan hasilnya
def pencatat_log(fungsi):
    def bungkus(*args, **kwargs):
        print(f"Memanggil {fungsi.__name__} dengan argumen: {args}, keyword argumen: {kwargs}")
        hasil = fungsi(*args, **kwargs)
        print(f"{fungsi.__name__} mengembalikan {hasil}")
        return hasil
    return bungkus

@pencatat_log
def tambah(a, b):
    return a + b

# Panggil fungsi
hasil = tambah(2, 3)


Memanggil tambah dengan argumen: (2, 3), keyword argumen: {}
tambah mengembalikan 5


In [20]:
hasil

5

# Intermediate

## Contoh 1: Mengatur Waktu Blok Kode dengan Manajer Konteks

Context manager ini mengukur waktu yang dibutuhkan oleh blok kode di dalam with. Membantu memonitor performa bagian tertentu dari program.

In [21]:
import time
from contextlib import contextmanager

# Context manager untuk mengukur waktu eksekusi sebuah blok kode
@contextmanager
def pengukur_waktu():
    waktu_mulai = time.time()
    yield  # Menjalankan blok kode di dalam 'with'
    waktu_selesai = time.time()
    print(f"Blok kode membutuhkan waktu {waktu_selesai - waktu_mulai} detik untuk dijalankan.")

# Menggunakan context manager
with pengukur_waktu():
    time.sleep(3)  # Contoh blok kode yang memerlukan waktu (tidur selama 3 detik)


Blok kode membutuhkan waktu 3.0001587867736816 detik untuk dijalankan.


## Contoh 2: Manajer Konteks Kustom untuk Manajemen Sumber Daya

Context manager ini menangani proses buka-tutup file secara otomatis. Menjamin file selalu ditutup meskipun terjadi error.

In [22]:
from contextlib import contextmanager

# Context manager untuk membuka dan menutup file secara otomatis
@contextmanager
def pembuka_berkas(nama_berkas, mode):
    try:
        berkas = open(nama_berkas, mode)
        yield berkas  # Memberikan kendali kepada blok 'with'
    finally:
        berkas.close()  # File akan ditutup secara otomatis meskipun terjadi error

# Menggunakan context manager untuk menulis ke dalam file
with pembuka_berkas("contoh.txt", "w") as berkas:
    berkas.write("Halo, Context Manager!")


# Advanced

## Contoh 1: Decorator Pencatatan Advanced dengan Fungsi signature

Decorator ini mencetak nama fungsi, argumen lengkap (baik posisi maupun keyword), dan hasil pengembalian. Versi logging yang lebih informatif.

In [23]:
# Decorator untuk mencatat pemanggilan fungsi beserta argumen dan hasilnya
def decorator_log_lanjutan(fungsi):
    def bungkus(*args, **kwargs):
        # Menggabungkan argumen posisi (args) menjadi string
        args_str = ', '.join([str(arg) for arg in args])
        # Menggabungkan argumen kata kunci (kwargs) menjadi string
        kwargs_str = ', '.join([f"{key}={value}" for key, value in kwargs.items()])
        # Gabungkan semua argumen jika ada
        semua_argumen = ', '.join(filter(None, [args_str, kwargs_str]))

        hasil = fungsi(*args, **kwargs)
        print(f"Fungsi {fungsi.__name__}({semua_argumen}) menghasilkan {hasil}")
        return hasil
    return bungkus

# Fungsi untuk mengalikan dua angka
@decorator_log_lanjutan
def perkalian(x, y):
    return x * y

perkalian(4, 5)


Fungsi perkalian(4, 5) menghasilkan 20


20

## Contoh 2: Context Managers Advanced untuk Exception Handling

Context manager ini menangkap exception tertentu selama blok with dijalankan, dan mencetak pesan error. Membantu penanganan error yang lebih elegan.

In [24]:
from contextlib import contextmanager

# Context manager untuk menangani pengecualian (exception)
@contextmanager
def penangkap_eksepsi(jenis_eksepsi):
    try:
        yield
    except jenis_eksepsi as e:
        print(f"Eksepsi tertangkap: {e}")

# Contoh penggunaan: mencoba pembagian dengan nol yang akan menimbulkan ZeroDivisionError
with penangkap_eksepsi(ZeroDivisionError):
    hasil = 10 / 0


Eksepsi tertangkap: division by zero


### Logging di Aplikasi Web (Flask)

Kasus: Memantau request API untuk debugging dan analisis.

Setiap request API ke /home akan dicatat waktu dan metodenya.

Dapat digunakan di berbagai endpoint API tanpa menulis ulang kode logging.

In [25]:
from flask import Flask, request
import datetime

app = Flask(__name__)

# decorator untuk mencatat permintaan HTTP yang masuk
def pencatat_permintaan(fungsi):
    def bungkus(*args, **kwargs):
        print(f"[{datetime.datetime.now()}] Permintaan ke {request.path} dengan metode {request.method}")
        return fungsi(*args, **kwargs)
    return bungkus

@app.route("/beranda", methods=["GET"])
@pencatat_permintaan
def beranda():
    return "Selamat datang di Halaman Beranda"

if __name__ == "__main__":
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\Users\LENOVO\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


### Caching dengan Decorators

Kasus: Mempercepat respon API dengan menyimpan hasil perhitungan.

Jika fungsi sudah pernah dijalankan, hasilnya disimpan di cache.

Mempercepat aplikasi, terutama jika perhitungan mahal (misalnya query database atau pemrosesan gambar).


In [26]:
import functools

# Penyimpanan hasil perhitungan
cache = {}

# decorator untuk menyimpan hasil fungsi agar tidak dihitung ulang
def decorator_caching(fungsi):
    @functools.wraps(fungsi)
    def bungkus(*args):
        if args in cache:
            print("Mengambil dari cache...")
            return cache[args]
        hasil = fungsi(*args)
        cache[args] = hasil
        return hasil
    return bungkus

@decorator_caching
def perhitungan_berat(x):
    print("Sedang menghitung...")
    return x ** 2  # Simulasi perhitungan yang memakan waktu.

print(perhitungan_berat(5))  # Pertama kali: Hitung
print(perhitungan_berat(5))  # Kedua kali: Ambil dari cache


Sedang menghitung...
25
Mengambil dari cache...
25


In [27]:
print(perhitungan_berat(10))
print(perhitungan_berat(2))

Sedang menghitung...
100
Sedang menghitung...
4


### Manajemen Koneksi Database (SQLAlchemy)

Kasus: Menjamin bahwa koneksi database ditutup setelah digunakan.

Context manager memastikan koneksi database selalu ditutup setelah digunakan.

Mencegah kebocoran koneksi database, yang bisa menyebabkan crash pada sistem.

In [28]:
from sqlalchemy import create_engine, text  # Tambahkan 'text'
from sqlalchemy.orm import sessionmaker
from contextlib import contextmanager

# Inisialisasi koneksi ke database SQLite
DATABASE_URL = "sqlite:///example.db"
engine = create_engine(DATABASE_URL)
SessionLocal = sessionmaker(bind=engine)

# Context manager untuk sesi database
@contextmanager
def get_db_session():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()  # Pastikan koneksi selalu ditutup

# Menggunakan sesi untuk menjalankan query
with get_db_session() as session:
    result = session.execute(text("SELECT sqlite_version();"))  # Bungkus query dengan text()
    print(result.fetchall())


[('3.45.3',)]


### Mengelola Koneksi API dengan requests

Kasus: Mengambil data dari API eksternal dengan koneksi yang aman.

Context manager memastikan koneksi API tertutup setelah digunakan.

Menghindari kebocoran sesi HTTP, yang bisa menghabiskan sumber daya server.

In [29]:
import requests
from contextlib import contextmanager

# Context manager untuk membuka sesi API
@contextmanager
def buka_sesi_api(url):
    sesi = requests.Session()
    try:
        respons = sesi.get(url)  # Mengirim permintaan GET ke URL
        yield respons  # Mengembalikan respons dari API
    finally:
        sesi.close()  # Menutup sesi setelah selesai

# URL API contoh
URL_API = "https://jsonplaceholder.typicode.com/todos/1"

# Menggunakan context manager untuk mengambil data dari API
with buka_sesi_api(URL_API) as respons:
    data = respons.json()
    print(data)

{'userId': 1, 'id': 1, 'title': 'delectus aut autem', 'completed': False}
